In [2]:
## Importando as libs necessárias
from google.colab import files
import os, io, time

# Criando o diretório inicial: /content/
os.chdir('/content/')
try:
  os.mkdir('Imagens', )
except:
  print('A pasta já existe.')
os.chdir('./Imagens')
os.listdir()
uploaded_images = files.upload()

Saving Bengal_1.jpg to Bengal_1.jpg
Saving Bengal_2.jpg to Bengal_2.jpg
Saving Bengal_3.jpg to Bengal_3.jpg
Saving Bengal_4.jpg to Bengal_4.jpg
Saving Bengal_5.jpg to Bengal_5.jpg
Saving Bengal_6.jpg to Bengal_6.jpg
Saving Bengal_7.jpg to Bengal_7.jpg
Saving Bengal_8.jpg to Bengal_8.jpg
Saving Bengal_9.jpg to Bengal_9.jpg
Saving Bengal_10.jpg to Bengal_10.jpg


In [4]:
## Reduzir as imagens
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.io import imread

# Local Binary Patterns
from skimage.feature import local_binary_pattern

import numpy as np
import pandas as pd

## Suprimindo Warnings
import warnings
warnings.filterwarnings('ignore')

lbp_features = []
radius = 3
n_points = 8 * radius

for filename in uploaded_images.keys():
  image = imread(filename)
  image_resized = resize(image, (256, 256), anti_aliasing=True)
  gray = rgb2gray(image_resized)

  # Aplicar LBP
  lbp = local_binary_pattern(gray, n_points, radius, method="uniform")

  # Criar histograma (descritor final)
  (hist, _) = np.histogram(lbp.ravel(),
                             bins=np.arange(0, n_points + 3),
                             range=(0, n_points + 2))

  # Normalizar histograma
  hist = hist.astype("float")
  hist /= (hist.sum() + 1e-7)

  lbp_features.append(hist)


# Substituindo as raças de cachorros e gatos por 'dog' e 'cat'
classe = ['dog' if 'american_bulldog' in file else 'cat' if 'Bengal' in file
           else file for file in uploaded_images.keys()]

## Adicionar as colunas com os valores do lbp + classe
data = {"lbp_features": lbp_features, "classe": classe}
df = pd.DataFrame(data)

# Transformando cada valor do lbp_feature em um atributo (coluna)
df2 = pd.DataFrame(df['lbp_features'].tolist())
df2.columns = df2.columns.map(lambda x: f'lbp_feature_{x+1}')
df = pd.concat([df.drop('lbp_features', axis=1), df2], axis=1)

# Mover a coluna 'classe' para o final
df['classe'] = df.pop('classe')

# Salvando csv resultante
df.to_csv('LBP_256_3r.csv', sep=',', index=False)
